In [9]:
from llama_index.llms.ollama import Ollama
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=600
)

In [10]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
mcp_client= BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools=McpToolSpec(client=mcp_client)

In [11]:
tools= await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

play_movie 
    Play a movie on the specified video streaming platform. If any parameter is missing ask the user for it one by one. Do not assume anything on your own

    Args:
        movie_name (str): Name or partial name of the movie to play.
        platform_name (str): Must be one of: Netflix, Hulu, Disney+, Amazon Prime, HBO Max, YouTube, Apple TV+.

    Returns:
        str: Confirmation message with matched movie (and rating if available), 
             or validation/suggestion message with other movie recommendations.
    
play_music 
    Play a song using Jio Saavn. If the song name is missing ask the user for it.

    Args:
        song_name (str): Name or partial name of the song to play.

    Returns:
        str: Confirmation message with matched song; prompts user if song name is not provided.
    
play_game 
    Launch a game using Jio Games service. If the name of the game is missing ask the user fro it.

    Args:
        game_name (str): Name or partial name of the 

In [12]:
SYSTEM_PROMPT="""\
    You are an AI assistant for tool calling for a smart TV control.
    Work with tools to perform the required actions and ask the user for any missing parameters in natural language if required but only 1 at a time.

"""

In [13]:
from llama_index.core.agent.workflow import FunctionAgent
async def get_agent(tools: McpToolSpec):
    tools=await tools.to_tool_list_async()
    agent=FunctionAgent(
        name="Agent",
        description="An agent interacting with a smart TV",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    print("agent returned successfuly")
    return agent


In [14]:
from llama_index.core.agent.workflow import FunctionAgent,ToolCallResult,ToolCall
from llama_index.core.workflow import Context

async def handle_user_message(
    message_content:str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose:bool = False,
):
    handler=agent.run(message_content,ctx=agent_context)
    print("agent ran")
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")
    response = await handler
    return str(response)

In [15]:
agent= await get_agent(mcp_tools)
agent_context=Context(agent)

agent returned successfuly


In [16]:
while True:
    user_input=input("You:")
    if(user_input.lower()=="exit"):
        break
    print("You:",user_input)
    response=await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent:", response)

You: i want to watch spiderman on netflix
agent ran
Calling tool play_movie with kwargs {'movie_name': 'Spiderman', 'platform_name': 'Netflix'}
Tool play_movie returned meta=None content=[TextContent(type='text', text="Playing 'Spider-Man' on Netflix", annotations=None, meta=None)] structuredContent={'result': "Playing 'Spider-Man' on Netflix"} isError=False


CancelledError: 